In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


/Users/lucytuan/anaconda3/envs/IDS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cat = pd.read_csv("../data/tmp.csv")

In [3]:
epoch = 100
batchsize = 250
lr = 1e-4
numworker =24

In [4]:
class predict(nn.Module):
    def __init__(self, infeature, outfeature):
        super().__init__()
        self.transform = nn.Linear(infeature, outfeature)
        self.leakyrelu = nn.LeakyReLU()
    def forward(self, x):
        x = self.transform(x)
        out = self.leakyrelu(x)
        return out

In [5]:
mymodel = predict(163, 1)
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
device = torch.device(device)

In [6]:
# news_feature:
"""
day 1 : 16*1 + 16*1 + ... = 160*1
day 2 : 16*1 + 16*1 + ... = 160*1
.
.
.
day 6643 : 16*1 + 16*1 + ... = 160*1

"""
# price_return:
"""
day 1 : 1*1 + 1*1 + 1*1 = 3*1
day 2 : 1*1 + 1*1 + 1*1 = 3*1
.
.
.
day 6643 : 1*1 + 1*1 + 1*1 = 3*1
"""
# ground_truth:
"""
day 1 : 1*1
day 2 : 1*1
.
.
.
day 6643 : 1*1
"""
news_feature = np.random.randn(6643, 160)
price_return = np.random.randn(6643,3)
ground_truth = np.random.randn(6643,1)

In [14]:
df = pd.read_csv("../data/new_pred.csv")


In [15]:
meta = df[df["symbol"]=="META"]

In [20]:
hi = meta["f01"]
print(hi)

0       -1.918452
1       -1.622287
2       -1.872377
3       -1.693291
4       -1.694585
           ...   
16021   -1.674121
16107   -1.683615
16306   -1.739593
16307   -1.689846
16308   -1.734896
Name: f01, Length: 1484, dtype: float64


In [21]:

print(meta)

KeyError: 'f01'

In [95]:
class Stock(Dataset):
    def __init__(self, news_feature, model, max_len) -> None:
        self.df = news_feature
        self.news = [float(score) for score in df["score"]]
        self.price = [self.tokenizer(text,padding='max_length', 
                       max_length = max_len, 
                       truncation=True,
                       return_tensors="pt") for text in df["content"]]
        
    def __getitem__(self, idx):
        return self.score[idx], self.content[idx]

    def __len__(self):
        return self.df.shape[0]

      symbol            datentime  \
0       META  2022-11-05 23:13:24   
1       META  2022-11-05 21:02:00   
2       META  2022-11-05 21:00:00   
3       META  2022-11-05 19:49:01   
4       META  2022-11-05 14:58:00   
...      ...                  ...   
25273   NFLX  2021-01-22 17:55:08   
25274   NFLX  2021-01-22 17:48:47   
25275   NFLX  2021-01-22 17:42:59   
25276   NFLX  2021-01-22 16:55:04   
25277   NFLX  2021-01-22 16:30:00   

                                                 content      match  \
0      Twitter co-founder Jack Dorsey admits he grew ...  13.746987   
1      The Fall of Big Tech Is Boosting Stock Quants ...   9.408848   
2      Meta Stock: Mr. Market Shorted Zuckerberg; Fut...  13.847881   
3      What stock investors should watch for in the m...  11.597897   
4                    Meta Lesson 1: Corporate Governance   3.993389   
...                                                  ...        ...   
25273  UBS Upgrades Disney On Streaming Strength, Val...  

In [91]:
# input dim : news_feature 160*1 + price_return_3_days 3*1 + ans 1*1 = 164*1
# ground_truth : price_return_next_day
news_feature = torch.tensor(news_feature)  # 16*10
price_return = torch.tensor(price_return)
ground_truth = torch.tensor(ground_truth)
# 80, 10, 10
lenn = len(news_feature)
lenp = len(price_return)
leng = len(ground_truth)
train_loader = torch.cat((news_feature[0:int(lenn*0.8),:], price_return[0:int(lenp*0.8),:], ground_truth[0:int(leng*0.8),:]),1)
valid_loader = torch.cat((news_feature[int(lenn*0.8):int(lenn*0.9),:], price_return[int(lenp*0.8):int(lenp*0.9),:], ground_truth[int(leng*0.8):int(leng*0.9),:]),1)
test_loader = torch.cat((news_feature[int(lenn*0.9):,:], price_return[int(lenp*0.9):,:], ground_truth[int(leng*0.9):,:]),1)

ValueError: could not determine the shape of object type 'DataFrame'

In [51]:
optimizer = torch.optim.SGD(mymodel.parameters(), lr=0.1, momentum=0.9)
lr_schedule = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.5)

In [ ]:
min_loss = 1e4

for eps in range(epoch):
    mymodel.train()
    loss_list = []
    for data in train_loader:
        optimizer.zero_grad()
        out = mymodel.forward(data[:len(data)-1])
        loss = nn.CrossEntropyLoss()
        loss = loss(out,data[len(data)-1])
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        loss_list = loss_list.append(loss)
    train_loss = np.mean(loss_list)
    mymodel.eval()
    valid_loss_list = []
    for news, price, ans in valid_loader:
        optimizer.zero_grad()
        out = mymodel.forward(news, price)
        loss = nn.CrossEntropyLoss()
        loss = loss(out,ans)
        if min_loss > loss:
            min_loss = loss
            torch.save(mymodel.state_dict(), f"./ckpt/mymodel.pt")
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        valid_loss_list = valid_loss_list.append(loss)
    valid_loss = np.mean(valid_loss_list)
    print(f"epoch:{eps}, train loss: {train_loss}, valid loss: {valid_loss}")

SyntaxError: unexpected EOF while parsing (1790028893.py, line 10)

In [ ]:
# test
mymodel.load_state_dict(torch.load(f"./ckpt/mymodel.pt"))
mymodel.eval()
for eps in range(epoch):
    loss_list = []
    acc_list = []
    for news, price, ans in iter(train_loader):
        optimizer.zero_grad()
        out = mymodel.forward(news, price)
        loss = nn.CrossEntropyLoss(out, ans)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        loss_list = loss_list.append(loss)
    test_loss = np.mean(loss_list)
print(f"test_loss:{test_loss}")

In [ ]:
aapl = cat[cat["symbol"]=="AAPL"] 
print(len(aapl)) #6643
print(aapl)
amzn = cat[cat["symbol"]=="AMZN"] 
print(len(amzn)) #7482
print(amzn)
nflx = cat[cat["symbol"]=="NFLX"] 
print(len(nflx)) # 5358
print(nflx)
googl = cat[cat["symbol"]=="GOOGL"] 
print(len(googl))  #4311
print(googl)
meta = cat[cat["symbol"]=="META"] 
print(len(meta))   # 1484
print(meta)